In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image as IPImage
import scipy.ndimage as ndimage
from scipy import signal, misc
from skimage import feature
import math

In [2]:
def showImage(im):
    plt.clf()
    plt.imshow(im)
    plt.show()

In [3]:
def roundAngle(angle_array):
    for i in range(angle_array.shape[0]):
        for j in range(angle_array.shape[1]):
            x = angle_array[i, j]
            if -22.5 < x < 22.5 or x > 157.5 or x < -157.5:
                angle_array[i, j] = 0
            elif 22.5 <= x <= 67.5 or -157.5 <= x <= -112.5:
                angle_array[i, j] = 45
            elif 67.5 < x < 112.5 or -112.5 < x < -67.5:
                angle_array[i, j] = 90
            else:
                angle_array[i, j] = 135
    return angle_array
        

In [4]:
def supressNonMax(im, thet):
    for i in range(im.shape[0]):
        for j in range(im.shape[1]):
            if thet[i, j] == 0:
                if i == 0:
                    if im[i, j] < im[i+1, j]:
                        im[i, j] = 0
                elif i == im.shape[0]-1:
                    if im[i, j] < im[i-1, j]:
                        im[i, j] = 0
                else:
                    if im[i, j] < im[i-1, j] or im[i, j] < im[i+1, j]:
                        im[i, j] = 0
            elif thet[i, j] == 90:
                if j == 0:
                    if im[i, j] < im[i, j+1]:
                        im[i, j] = 0
                elif j == im.shape[1]-1:
                    if im[i, j] < im[i, j-1]:
                        im[i, j] = 0
                else:
                    if im[i, j] < im[i, j+1] or im[i, j] < im[i, j-1]:
                        im[i, j] = 0
            elif thet[i, j] == 45:
                if i == 0:
                    if j != 0:
                        if im[i, j] < im[i+1, j-1]:
                            im[i, j] = 0
                    continue
                if j == 0:
                    if i != 0:
                        if im[i, j] < im[i-1, j+1]:
                            im[i, j] = 0
                    continue
                if i == im.shape[0]-1:
                    if j != im.shape[1]-1:
                        if im[i, j] < im[i-1, j+1]:
                            im[i, j] = 0
                    continue
                if j == im.shape[1]-1:
                    if i != im.shape[0]-1:
                        if im[i, j] < im[i+1, j-1]:
                            im[i, j] = 0
                    continue
                else:
                    if im[i, j] < im[i-1, j+1] or im[i, j] < im[i+1, j-1]:
                        im[i, j] = 0
            elif thet[i, j] == 135:
                if i == 0:
                    if j != im.shape[1]-1:
                        if im[i, j] < im[i+1, j+1]:
                            im[i, j] = 0
                    continue
                if j == 0:
                    if i != im.shape[0]-1:
                        if im[i, j] < im[i+1, j+1]:
                            im[i, j] = 0
                    continue
                if i == im.shape[0]-1:
                    if j != 0:
                        if im[i, j] < im[i-1, j-1]:
                            im[i, j] = 0
                    continue
                if j == im.shape[1]-1:
                    if i != 0:
                        if im[i, j] < im[i-1, j-1]:
                            im[i, j] = 0
                    continue
                else:
                    if im[i, j] < im[i-1, j-1] or im[i, j] < im[i+1, j+1]:
                        im[i, j] = 0

    return im
            

In [5]:
def thresholding(im, high):
    for i in range(im.shape[0]):
        for j in range(im.shape[1]):
            x = im[i, j]
            if x > high:
                im[i, j] = 1
            else:
                im[i, j] = 0
    return im
    

In [6]:
filename = 'Test2.jpg'
name = 'Test2'

In [7]:
actual_image = misc.imread(filename, True)
image = misc.imread(filename, True)
blurred_image = ndimage.gaussian_filter(image, sigma=1)

In [8]:
image.shape

(128, 128)

In [9]:
sobel_x = np.array([[-1, 0,  1], [-2, 0, 2], [-1, 0,  1]])
sobel_y = np.array([[1, 2,  1], [0, 0, 0], [-1, -2,  -1]])

In [10]:
image_gx = signal.convolve2d(blurred_image, sobel_x, mode='same', boundary='symm')
image_gy = signal.convolve2d(blurred_image, sobel_y, mode='same', boundary='symm')

In [11]:
image_gx.shape

(128, 128)

In [12]:
image_gradient = np.sqrt(np.multiply(image_gx, image_gx) + np.multiply(image_gy, image_gy))
image_theta = np.arctan2(image_gx, image_gy) * 180 / np.pi

In [13]:
round_theta = np.copy(image_theta)
round_theta = roundAngle(round_theta)

In [14]:
image_suppressed = np.copy(image_gradient)
image_suppressed = supressNonMax(image_suppressed, round_theta)

In [15]:
image_all_edge = np.copy(image_suppressed)
image_all_edge = thresholding(image_all_edge, 0)
misc.imsave(name + '_all.jpg', image_all_edge)

In [16]:
threshold = 1.33*np.median(image)
image_thresholded = np.copy(image_suppressed)
image_thresholded = thresholding(image_thresholded, threshold)
misc.imsave(name + '_threshold.jpg', image_thresholded)
# showImage(image_thresholded)

In [17]:
def hough(im, r):
#     plt.clf() # clear the figure
    hough_bins = np.zeros((im.shape[0], im.shape[1]))
    for x in range(im.shape[0]):
        for y in range(im.shape[1]):
            if im[x,y] == 1:
#                 hough_x = ([])
#                 hough_y = ([])
                for x0 in range(im.shape[0]):
                    if r*r - (x-x0)*(x-x0) < 0:
                        continue
                    y0 = y - math.sqrt(r*r - (x-x0)*(x-x0))
                    hough_bins[x0, int(round(y0))] = hough_bins[x0, int(round(y0))] + 1
#                     hough_x = np.append(hough_x, x0)
#                     hough_y = np.append(hough_y, y0)
#                 p = np.polyfit(hough_x,hough_y,2)
#                 plt.plot(hough_x,np.polyval(p,hough_x),'r-')
#     plt.show()
    return hough_bins

In [18]:
hb = hough(image_thresholded, 20)

In [19]:
def storeBestCircle(hough, number):
    bestCircles = ([])
    
    for x in range(number):
        loc = np.where(hough == hough.max())
        circle = [loc[1][0], loc[0][0]]
        bestCircles = np.append(bestCircles, circle)
        hough[loc[0][0], loc[1][0]] = 0
        
    bestCircles.shape = (number, 2)
    return bestCircles
        

In [20]:
hb2 = np.copy(hb)
best = storeBestCircle(hb2, 5)

In [21]:
def drawCircle(centers, r):
    plt.clf()
    plt.imshow(image_thresholded)
    for center in centers:
        circle1 = plt.Circle((center[0], center[1]),r,color='b',fill=0, linewidth=3)
        plt.gca().add_artist(circle1)
    plt.show()

In [22]:
drawCircle(best, 20)

In [23]:
def houghSansRadius(im):
    im_diagonal = math.sqrt(im.shape[0]*im.shape[0] + im.shape[1]*im.shape[1])
    hough_bins = np.zeros((im.shape[0], im.shape[1], int(math.ceil(im_diagonal))))
    for x in range(im.shape[0]):
        for y in range(im.shape[1]):
            if im[x,y] == 1:
                for x0 in range(im.shape[0]):
                        for y0 in range(im.shape[1]):
                            domain = (x-x0)*(x-x0) + (y-y0)*(y-y0)
                            if domain < 0:
                                continue
                            r = math.sqrt(domain)
                            hough_bins[x0, y0, int(round(r))] = hough_bins[x0, y0, int(round(r))] + 1
            if x % 50 == 0 and y % 50 == 0:
                print ([x, y])
    return hough_bins

In [24]:
hsr = houghSansRadius(image_thresholded)

[0, 0]
[0, 50]
[0, 100]
[50, 0]
[50, 50]
[50, 100]
[100, 0]
[100, 50]
[100, 100]


In [25]:
hsr.max()

121.0

In [26]:
np.where(hsr == hsr.max())

(array([62]), array([67]), array([59]))

In [27]:
len(hsr)

128

In [28]:
len(hsr[0])

128

In [29]:
len(hsr[0][0])

182

In [30]:
def storeBestCircleRadius(hough, number):
    bestCircles = ([])
    
    for x in range(number):
        loc = np.where(hough == hough.max())
        circle = [loc[1][0], loc[0][0], loc[2][0]]
        bestCircles = np.append(bestCircles, circle)
        hough[loc[0][0], loc[1][0], loc[2][0]] = 0
        
    bestCircles.shape = (number, 3)
    return bestCircles

In [31]:
hsr2= np.copy(hsr)

In [32]:
best_hsr = storeBestCircleRadius(hsr, 5)

In [35]:
def drawCircleRadius(centers):
    plt.clf()
    plt.imshow(image_thresholded)
    for center in centers:
        print center
        circle1 = plt.Circle((center[0], center[1]),center[2],color='b',fill=0, linewidth=3)
        plt.gca().add_artist(circle1)
    plt.show()

In [36]:
drawCircleRadius(best_hsr)

[ 67.  62.  59.]
[ 67.  63.  59.]
[ 67.  61.  59.]
[ 64.  54.  62.]
[ 66.  62.  60.]


In [48]:
np.zeros(4.)

array([ 0.,  0.,  0.,  0.])

In [49]:
np.zeros(4)

array([ 0.,  0.,  0.,  0.])